<a href="https://colab.research.google.com/github/hardlyrichie/data-science-gss/blob/master/clean_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cleaning Data

This notebook reads in the gss strata data and packages it into a more managable hdf5 file. The motivation behind putting this in a separate notebook is that for stratified sampling to work, I had to copy the weights (`wtssall`) from Allen's gss data. When I export the `wtssall` data from my gss variables, they are all `NaN`. I have tried debugging for a long time, but couldn't get any results. Therefore, I opted to replace my `wtssall` data with Allen's.

## Reading in data

In [0]:
import pandas as pd
import numpy as np

In [0]:
import re

def read_stata_dict(fp, **options):
    """Reads a Stata dictionary file.

    fp: open file pointer
    options: dict of options passed to open()

    returns: DataFrame
    """
    type_map = dict(byte=int, int=int, long=int, float=float,
                    double=float, numeric=float)

    var_info = []
    for line in fp:
        match = re.search(r'_column\(([^)]*)\)', line)
        if not match:
            continue
        start = int(match.group(1))
        t = line.split()
        vtype, name, fstring = t[1:4]
        name = name.lower()
        if vtype.startswith('str'):
            vtype = str
        else:
            vtype = type_map[vtype]
        long_desc = ' '.join(t[4:]).strip('"')
        var_info.append((start, vtype, name, fstring, long_desc))

    columns = ['start', 'type', 'name', 'fstring', 'desc']
    variables = pd.DataFrame(var_info, columns=columns)

    # fill in the end column by shifting the start column
    # NOTE: the last column doesn't work
    variables['end'] = variables.start.shift(-1, fill_value=0)

    return variables

In [3]:
# Load the data file

import os

if not os.path.exists('gss_eda.tar.gz'):
    !wget https://github.com/AllenDowney/PoliticalAlignmentCaseStudy/raw/master/gss_eda.tar.gz
    !tar -xzf gss_eda.tar.gz

--2020-05-04 05:37:09--  https://github.com/AllenDowney/PoliticalAlignmentCaseStudy/raw/master/gss_eda.tar.gz
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/AllenDowney/PoliticalAlignmentCaseStudy/master/gss_eda.tar.gz [following]
--2020-05-04 05:37:11--  https://raw.githubusercontent.com/AllenDowney/PoliticalAlignmentCaseStudy/master/gss_eda.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7147633 (6.8M) [application/octet-stream]
Saving to: ‘gss_eda.tar.gz’

gss_eda.tar.gz      100%[===================>]   6.82M  15.8MB/s    in 0.4s    

2020-05-04 05:37:13 (15.8 MB/s) - ‘gss_eda.tar.gz’ sav

In [0]:
with open('GSS.dct') as fp:
    variables = read_stata_dict(fp)
colspecs = variables[['start', 'end']]
names = variables['name']
with open('GSS.dat') as fp:
    gss = pd.read_fwf(fp,
                      colspecs=colspecs.values.tolist(),
                      names=names)
allen_weights = gss['wtssall']

In [5]:
if not os.path.exists('GSS.tar.gz'):
    !wget https://github.com/hardlyrichie/data-science-gss/raw/master/GSS.tar.gz
    !tar -xzf GSS.tar.gz

--2020-05-04 05:37:28--  https://github.com/hardlyrichie/data-science-gss/raw/master/GSS.tar.gz
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hardlyrichie/data-science-gss/master/GSS.tar.gz [following]
--2020-05-04 05:37:28--  https://raw.githubusercontent.com/hardlyrichie/data-science-gss/master/GSS.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 453489 (443K) [application/octet-stream]
Saving to: ‘GSS.tar.gz’

GSS.tar.gz          100%[===================>] 442.86K  --.-KB/s    in 0.07s   

2020-05-04 05:37:30 (5.87 MB/s) - ‘GSS.tar.gz’ saved [453489/453489]



In [0]:
with open('GSS.dct') as fp:
    variables = read_stata_dict(fp)
colspecs = variables[['start', 'end']]
names = variables['name']
with open('GSS.dat') as fp:
    gss = pd.read_fwf(fp,
                      colspecs=colspecs.values.tolist(),
                      names=names)

As you can see, my `wtssall` column is just `NaN`. 

In [7]:
gss.tail()

,year,id_,rincome,relig,happy,mntlhlth,satjob1,ballot,wtssall
64809,2018,2344,13,1,2,0,2,1,NaN
64810,2018,2345,10,1,2,0,1,1,NaN
64811,2018,2346,3,3,1,-1,0,3,NaN
64812,2018,2347,0,1,1,-1,0,2,NaN
64813,2018,2348,0,2,1,-1,0,1,NaN


In [8]:
gss['wtssall'] = allen_weights
gss.tail()

,year,id_,rincome,relig,happy,mntlhlth,satjob1,ballot,wtssall
64809,2018,2344,13,1,2,0,2,1,0.471499
64810,2018,2345,10,1,2,0,1,1,0.942997
64811,2018,2346,3,3,1,-1,0,3,0.942997
64812,2018,2347,0,1,1,-1,0,2,0.942997
64813,2018,2348,0,2,1,-1,0,1,0.471499


## Perform stratified resampling

In [0]:
def resample_rows_weighted(df, column):
    """Resamples a DataFrame using probabilities proportional to given column.

    df: DataFrame
    column: string column name to use as weights

    returns: DataFrame
    """
    weights = df[column]
    sample = df.sample(n=len(df), replace=True, weights=weights)
    return sample

In [0]:
def resample_by_year(df, column):
    """Resample rows within each year.

    df: DataFrame
    column: string name of weight variable

    returns DataFrame
    """
    grouped = df.groupby('year')
    samples = [resample_rows_weighted(group, column)
               for _, group in grouped]
    sample = pd.concat(samples, ignore_index=True)
    return sample

In [0]:
np.random.seed(21)
sample = resample_by_year(gss, 'wtssall')

## Save results into HDF5 file

In [0]:
gss.to_hdf('gss.hdf5', 'gss', complevel=3)